In [1]:
import matplotlib.pyplot as plt
import scipy.io
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from yad2k.models.keras_yolo import yolo_head
from tensorflow.python.framework.ops import EagerTensor

from yolo_utils import draw_boxes,get_colors_for_classes, scale_boxes, read_classes, read_anchors, preprocess_image

In [2]:
def yolo_filter_boxes(boxes,box_confidence,box_class_probs,threshold=0.6):
    '''
    boxes= 19,19,5,4 - shape of box (bx,by,bh,bw)
    box_confidence = 19,19,5,1 - pc of box of each anchor
    box_class_probs = 19,19,5,80 - c0 to c80, 80 dimensional probabi;ity that the grid contains a certain class of object in that anchor box

    Returns -
    scores - tensor of shape (None,), containing class probability score for selected boxes
    boxes - tensor of shape (None,4), containing (b_x,b_y,b_h,b_w) coordinates of selected boxes
    classes - tensor of shape (None,) containing index of the class detected by tthe selected box
    '''

    # x=10
    # y=tf.constant(100)

    #compute box_scores
    box_scores=box_confidence*box_class_probs

    #Find the box_classes using the max box_scores, keep track of the coreresponsidng scores
    box_classes=tf.math.argmax(box_scores,axis=-1)
    box_class_scores=tf.math.reduce_max(box_scores,axis=-1)

    #Create a filtering mask based on "box_class_scores" by threshold, True for boxes to keep
    filtering_mask = (box_class_scores>=threshold)

    #Apply the mask
    scores=tf.boolean_mask(box_class_scores,filtering_mask)
    boxes=tf.boolean_mask(boxes,filtering_mask)
    classes=tf.boolean_mask(box_classes,filtering_mask)

    return scores, boxes, classes

In [10]:
tf.random.set_seed(10)
box_confidence = tf.random.normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1)
boxes = tf.random.normal([19, 19, 5, 4], mean=1, stddev=4, seed = 1)
box_class_probs = tf.random.normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1)
scores, boxes, classes = yolo_filter_boxes(boxes, box_confidence, box_class_probs, threshold = 0.5)
print("scores[2] = " + str(scores[2].numpy()))
print("boxes[2] = " + str(boxes[2].numpy()))
print("classes[2] = " + str(classes[2].numpy()))
print("scores.shape = " + str(scores.shape))
print("boxes.shape = " + str(boxes.shape))
print("classes.shape = " + str(classes.shape))

assert type(scores) == EagerTensor, "Use tensorflow functions"
assert type(boxes) == EagerTensor, "Use tensorflow functions"
assert type(classes) == EagerTensor, "Use tensorflow functions"

assert scores.shape == (1789,), "Wrong shape in scores"
assert boxes.shape == (1789, 4), "Wrong shape in boxes"
assert classes.shape == (1789,), "Wrong shape in classes"

assert np.isclose(scores[2].numpy(), 9.270486), "Values are wrong on scores"
assert np.allclose(boxes[2].numpy(), [4.6399336, 3.2303846, 4.431282, -2.202031]), "Values are wrong on boxes"
assert classes[2].numpy() == 8, "Values are wrong on classes"


scores[2] = 9.270486
boxes[2] = [ 4.6399336  3.2303846  4.431282  -2.202031 ]
classes[2] = 8
scores.shape = (1789,)
boxes.shape = (1789, 4)
classes.shape = (1789,)


Even after this, we would require non max suppression, finding IOU first

In [3]:
def iou (box1,box2):
    '''
    Implemet the intersection over union between box1 and box2

    Arguments:
    bpx1 -- forst box, list object with cordinates (box1_x1,box1_y1,box1_x2,box1_y2)
    box2-- second box, list object with cordinates (box2_x1,box2_y1,box2_x2,box2_y2)
    '''
    (x1_1,y1_1,x1_2,y1_2)=box1
    (x2_1,y2_1,x2_2,y2_2)=box2

    #calculate xi_1,yi_1,xi_2,yi_2
    xi_1=max(x1_1,x2_1)
    yi_1=max(y1_1,y2_1)
    xi_2=min(x1_2,x2_2)
    yi_2=min(y1_2,y2_2)

    inter_width=max(0,xi_2-xi_1)
    inter_height=max(0,yi_2-yi_1)
    inter_area=inter_height*inter_width

    #calculate union arera
    area_1=(x1_2-x1_1)*(y1_2-y1_1)
    area_2=(x2_2-x2_1)*(y2_2-y2_1)
    union_area=area_1+area_2-inter_area

    #compute IOU
    iou=inter_area/union_area

    return iou

In [4]:
## Test case 1: boxes intersect
box1 = (2, 1, 4, 3)
box2 = (1, 2, 3, 4)

print("iou for intersecting boxes = " + str(iou(box1, box2)))
assert iou(box1, box2) < 1, "The intersection area must be always smaller or equal than the union area."
assert np.isclose(iou(box1, box2), 0.14285714), "Wrong value. Check your implementation. Problem with intersecting boxes"

## Test case 2: boxes do not intersect
box1 = (1,2,3,4)
box2 = (5,6,7,8)
print("iou for non-intersecting boxes = " + str(iou(box1,box2)))
assert iou(box1, box2) == 0, "Intersection must be 0"

## Test case 3: boxes intersect at vertices only
box1 = (1,1,2,2)
box2 = (2,2,3,3)
print("iou for boxes that only touch at vertices = " + str(iou(box1,box2)))
assert iou(box1, box2) == 0, "Intersection at vertices must be 0"

## Test case 4: boxes intersect at edge only
box1 = (1,1,3,3)
box2 = (2,3,3,4)
print("iou for boxes that only touch at edges = " + str(iou(box1,box2)))
assert iou(box1, box2) == 0, "Intersection at edges must be 0"

print("\033[92m All tests passed!")

iou for intersecting boxes = 0.14285714285714285
iou for non-intersecting boxes = 0.0
iou for boxes that only touch at vertices = 0.0
iou for boxes that only touch at edges = 0.0
 All tests passed!


Now ready to implement non-max suppression

In [5]:
def yolo_non_max_suppression(scores,boxes,classes,max_boxes=10,iou_threshold=0.5):
    '''
    scores- tensor of shaoe (None,), output of yolo-filter_boxes()
    boxes - tensor of shape (none,4), output of yolo_filter_boxes() that have been scaled to the image
    classes - tensor of shape (None,), output of yolo_filter_boxes()
    max_boxes=maximum number of predicted boxes you'd like
    iou_threshold -- real value

    Returns:
    scores - tensor of shape (,None),predicted score for each box
    boxes -- tensor of shape (4,None), predictedd box cordinates
    classes - tensor of shape (,None), predicted class for each box

    the output will have scores, boxes and classes transposed for convenience
    '''

    max_boxes_tensor = tf.Variable(max_boxes,dtype='int32')

    #use tf.image.non_max_suppression() to get the list of indices corresponding to boxees you keep

    nms_indices=tf.image.non_max_suppression(boxes,scores,max_boxes_tensor,iou_threshold)

    #use tf.gather() to select only nms_indices
    scores=tf.gather(scores,nms_indices)
    boxes=tf.gather(boxes,nms_indices)
    classes=tf.gather(classes,nms_indices)

    return scores,boxes,classes

In [14]:
tf.random.set_seed(10)
scores = tf.random.normal([54,], mean=1, stddev=4, seed = 1)
boxes = tf.random.normal([54, 4], mean=1, stddev=4, seed = 1)
classes = tf.random.normal([54,], mean=1, stddev=4, seed = 1)
scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes)

assert type(scores) == EagerTensor, "Use tensoflow functions"
print("scores[2] = " + str(scores[2].numpy()))
print("boxes[2] = " + str(boxes[2].numpy()))
print("classes[2] = " + str(classes[2].numpy()))
print("scores.shape = " + str(scores.numpy().shape))
print("boxes.shape = " + str(boxes.numpy().shape))
print("classes.shape = " + str(classes.numpy().shape))

assert type(scores) == EagerTensor, "Use tensoflow functions"
assert type(boxes) == EagerTensor, "Use tensoflow functions"
assert type(classes) == EagerTensor, "Use tensoflow functions"

assert scores.shape == (10,), "Wrong shape"
assert boxes.shape == (10, 4), "Wrong shape"
assert classes.shape == (10,), "Wrong shape"

assert np.isclose(scores[2].numpy(), 8.147684), "Wrong value on scores"
assert np.allclose(boxes[2].numpy(), [ 6.0797963, 3.743308, 1.3914018, -0.34089637]), "Wrong value on boxes"
assert np.isclose(classes[2].numpy(), 1.7079165), "Wrong value on classes"

print("\033[92m All tests passed!")

scores[2] = 8.147684
boxes[2] = [ 6.0797963   3.743308    1.3914018  -0.34089637]
classes[2] = 1.7079165
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)
 All tests passed!


In [6]:
def yolo_boxes_to_corners (box_xy,box_wh):
    box_mins=box_xy-(box_wh/2.)
    box_maxes=box_xy+(box_wh/2.)

    return tf.keras.backend.concatenate([
        box_mins[...,1:2],# y_min
        box_mins[...,0:1], #x_min
        box_maxes[...,1:2],#y_max
        box_maxes[...,0:1] #x_max
    ])

In [9]:
def yolo_eval(yolo_outputs,image_shape=(720.0,1280.0),max_boxes=10,score_threshold=0.6,iou_threshold=0.5):

    box_xy,box_wh,box_confidence,box_class_probs=yolo_outputs
    # print(box_wh)
    
    boxes=yolo_boxes_to_corners(box_xy,box_wh)
    scores,boxes,classes=yolo_filter_boxes(boxes,box_confidence,box_class_probs,score_threshold)
    print(boxes.dtype)
    #scale boxes back to original image shape (720,1280 here)
    boxes = scale_boxes(boxes, image_shape)

    #Non-max
    scores,boxes,classes=yolo_non_max_suppression(scores,boxes,classes,max_boxes,iou_threshold)

    return scores,boxes,classes

In [10]:
tf.random.set_seed(10)
yolo_outputs = (tf.random.normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                tf.random.normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                tf.random.normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1),
                tf.random.normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1))
scores, boxes, classes = yolo_eval(yolo_outputs)
print("scores[2] = " + str(scores[2].numpy()))
print("boxes[2] = " + str(boxes[2].numpy()))
print("classes[2] = " + str(classes[2].numpy()))
print("scores.shape = " + str(scores.numpy().shape))
print("boxes.shape = " + str(boxes.numpy().shape))
print("classes.shape = " + str(classes.numpy().shape))

assert type(scores) == EagerTensor, "Use tensoflow functions"
assert type(boxes) == EagerTensor, "Use tensoflow functions"
assert type(classes) == EagerTensor, "Use tensoflow functions"

assert scores.shape == (10,), "Wrong shape"
assert boxes.shape == (10, 4), "Wrong shape"
assert classes.shape == (10,), "Wrong shape"
    
assert np.isclose(scores[2].numpy(), 171.60194), "Wrong value on scores"
assert np.allclose(boxes[2].numpy(), [-1240.3483, -3212.5881, -645.78, 2024.3052]), "Wrong value on boxes"
assert np.isclose(classes[2].numpy(), 16), "Wrong value on classes"
    
print("\033[92m All tests passed!")

<dtype: 'float32'>
tf.Tensor([[ 720. 1280.  720. 1280.]], shape=(1, 4), dtype=float32)
scores[2] = 171.60194
boxes[2] = [-1240.3483 -3212.5881  -645.78    2024.3052]
classes[2] = 16
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)
 All tests passed!
